In [86]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [87]:
data = pd.read_csv("abalone.csv")
data

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [88]:
corr = data.corr()
corr

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
Length,1.000000,0.986812,0.827554,0.925261,0.897914,0.903018,0.897706,0.556720
Diameter,0.986812,1.000000,0.833684,0.925452,0.893162,0.899724,0.905330,0.574660
Height,0.827554,0.833684,1.000000,0.819221,0.774972,0.798319,0.817338,0.557467
Whole weight,0.925261,0.925452,0.819221,1.000000,0.969405,0.966375,0.955355,0.540390
Shucked weight,0.897914,0.893162,0.774972,0.969405,1.000000,0.931961,0.882617,0.420884
Viscera weight,0.903018,0.899724,0.798319,0.966375,0.931961,1.000000,0.907656,0.503819
Shell weight,0.897706,0.905330,0.817338,0.955355,0.882617,0.907656,1.000000,0.627574
Rings,0.556720,0.574660,0.557467,0.540390,0.420884,0.503819,0.627574,1.000000


In [89]:
X = data[['Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight']].values  

Y = data['Rings'].values

In [90]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [107]:
class LinearLeastSquare:
    def __init__(self):
        pass
    
    def fit(self, X, Y):
        self.w = np.matmul(inv(np.matmul(X.T, X)), np.matmul(X.T, Y))
        
    def predict(self, x):
        rings_pred = np.matmul(x, self.w)
        return rings_pred
    
    def evaluate(self, X, Y, loss="MAE"):
        Y_pred = []
        for i in range(X.shape[0]):
            y_pred = self.predict(X[i])
            Y_pred.append(y_pred)
        
        Y_pred = np.array(Y_pred)
        Error = Y - Y_pred
        
        if loss == "MAE":
            return np.mean(np.abs(Error))
        elif loss == "MSE":
            return np.mean(Error ** 2)
        elif loss == "Huber":
            is_small_error = np.abs(Error) < 1
            sq = np.square(Error) / 2
            linear  = np.abs(Error) - 0.5
            hub_loss = np.where(is_small_error, sq, linear)
            return hub_loss
        elif loss == "Hinge":
            pred = np.array([-1 if i==0 else i for i in Y_pred])
            hinge_loss = np.mean([max(0, 1-x*y) for x, y in zip(Y, pred)])
            return hinge_loss

In [108]:
lls = LinearLeastSquare()

In [109]:
lls.fit(X_train, Y_train)

In [110]:
MAE_loss = lls.evaluate(X_test, Y_test, "MAE")
print("MAE loss = ", MAE_loss)

MAE loss =  1.6477846777548224


In [111]:
MSE_loss = lls.evaluate(X_test, Y_test, "MSE")
print("MSE loss = ", MSE_loss)

MSE loss =  5.128210208661466


In [112]:
hub_loss = lls.evaluate(X_test, Y_test, "Huber")
print("Huber loss = ", hub_loss)

Huber loss =  [7.05259860e-03 4.30192760e-01 4.92868680e+00 ... 1.90137532e-01
 6.28904283e-04 1.26820732e+00]


In [113]:
hinge_loss = lls.evaluate(X_test, Y_test, "Hinge")
print("Hinge loss = ", hinge_loss)

Hinge loss =  0.01346528955082542
